# Adding Badgeout Station to Stores

## Some definitions

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

In [ ]:
# For production
# namespace = "fcr-badgeout-fhrp869iwhncz8bimlgjbp3ddglhgvg6"
# machinaccess = False

# For testing
namespace = "ki"
machineaccess = True

# Get them from 1password (for testing only)
user = ""
password = ""

In [ ]:
class ShopperData:
    
    def __init__(self, namespace, machineaccess=False):
        if machineaccess:
            self._base_url = f"https://{namespace}-machineaccess.svcs-dev.gbo/shopperdata/api/v1"
        else: 
            self._base_url = f"https://{namespace}.svcs.gbo/shopperdata/api/v1"
        self._brands = []
    
    def _requests(self, method, url, payload=None):
        if method == "get":
            request_object = requests.get
        elif method == "put":
            request_object = requests.put
        elif method == "post":
            request_object = requests.post
        else:
            raise Exception("Unrecognized method")
        
        try:
            if payload is None:
                r = request_object(url, verify=False, auth=HTTPBasicAuth(user, password))
            else:
                r = request_object(url, verify=False, json=payload, auth=HTTPBasicAuth(user, password))
            r.raise_for_status()
            return r.json()
        except Exception as e:
            print(e)

    def _update_store_brands(self):
        """
        Grabs all store brands in ShopperDB and save it in memory.
        """
        r = self._requests("get", self._base_url + "/store_brands")
        try:
            self._brands = r['data']
        except Exception as e:
            self._brands = []
    
    def get_brand_by_name(self, name):
        """
        Get one particular store brand by its *real* name. Returns list.
        """
        if not self._brands:
            self._update_store_brands()
        return list(filter(lambda x: x["store_name"].lower() == name.lower(), self._brands))
    
    def get_all_stores(self, name):
        """
        Get all stores with store brand name.
        
        Returns a list of store_uuid and first line of street address
        """
        try:
            brand = self.get_brand_by_name(name)[0]
        except Exception as e:
            print("get all stores failed", e)
            print("No stores with the name. Did you use the correct name?")
            return []
        
        store_brand_uuid = brand["external_uuid"]
        try: 
            r = self._requests("get", self._base_url + f"/store_brands/{store_brand_uuid}/stores")
            return [{"store_uuid": store["external_uuid"], "address": store["address_line1"], "store_id": store["id"]} for store in r["data"]]
        except Exception as e:
            print(f"Something went wrong: {e}")
            return []
            
    def add_badgeout_station(self, store_uuid, badgeout_station_id, status_override="active"):
        """
        Adds a new badgeout station to the store. Will fail if badgeout_station_id is already in use. Use update in that case.
        """
        payload = dict(
            badgeout_station_id=badgeout_station_id,
            store_uuid=store_uuid,
            status_override=status_override
        )
        url = self._base_url + "/badgeout_stations/"
        return self._requests("post", url, payload=payload)

            
    def get_badgeout_station(self, badgeout_station_id):
        """
        Gets information about badgeout station. Useful to check if it got added properly.
        """
        return self._requests("get", self._base_url + f"/badgeout_stations/{badgeout_station_id}")
        
    def update_badgeout_station(self, store_uuid, badgeout_station_id, status_override="active"):
        """
        Updates existing badgeout station.
        """
        payload = dict(
            store_uuid=store_uuid,
            status_override=status_override
        )
        return self._requests("put", self._base_url + f"/badgeout_stations/{badgeout_station_id}", payload=payload)

## Get store_uuid

In [ ]:
shopper_data = ShopperData(namespace, machineaccess)
shopper_data.get_all_stores("")

## Add badgeout station

In [ ]:
shopper_data.add_badgeout_station("5c46abd4-0718-4930-9655-0c0ea47ae011", "fake-station") # Use update if fails

In [ ]:
shopper_data.get_badgeout_station("fake-station")